# Projeto 1 - Ciência dos Dados

Nome: José Rafael Martins Fernandes <br />
Nome: Lucca Barufatti Velini Sanches

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
from math import *
from emoji import UNICODE_EMOJI
from IPython.display import display
pd.options.display.max_rows = 13

In [15]:
#!pip install emoji

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [16]:
import os

filename = 'bigmac.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com o projeto!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo bigmac.xlsx, tudo certo para prosseguir com o projeto!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [17]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Classificação
0,a tristeza do domingo a noite me obrigada a pe...,1
1,"caralhooo, fiz o molho do big mac e ficou mto ...",0
2,rt @rodrigodasilva: essa aqui é a fila da aber...,0
3,sempre ele🥺🥰\n\n[15/09/2020 14:50:04] amor💗: v...,0
4,very depressed mas pelo menos tenho um big mac...,1


In [18]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Classificação
0,comer um big mac 🥺\nexperimentar perfume na re...,0
1,ontem eu pedi um big mac e não estava sentindo...,0
2,@jhseokjin e se fosse um big mac?,0
3,"ontem amassei um big mac, um cheddar e a batat...",0
4,rt @mcdonalds_br: ⏱contagem regressiva... é am...,0


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

ESCREVA AQUI...

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### Limpando bases de dados

In [19]:
#Função para limpar o texto
def is_emoji(s):
    return s in UNICODE_EMOJI

def limpa(texto):
    texto = texto.lower()
    pontuacao = r"[-'()'\"#/@*;:<>{}=~|.?,]"
    padrao = re.compile(pontuacao)
    texto = re.sub(padrao, "", texto)
    texto = re.sub('https', "", texto)
    texto = re.sub('\n', '', texto)
    texto = ''.join(' ' + char + ' ' if is_emoji(char) else char for char in texto).strip()
    texto = " ".join(texto.split())
    return texto

#Separando os tweets pela classificação
tweets_rel = train.loc[train['Classificação'] == 1, :]
tweets_irel = train.loc[train['Classificação'] == 0, :]

#Limpando os tweets da base de treinamento
tweets_rel_limpo = tweets_rel['Treinamento'].apply(limpa)
tweets_irel_limpo = tweets_irel['Treinamento'].apply(limpa)

#### Tweets relevantes

In [20]:
lista_tweets_rel_completos = [] #Lista com cada tweet relevante dentro de uma lista

for tweet in tweets_rel_limpo:
    lista_tweets_rel_completos.append(tweet.split()) 

palavras_tweets_rel = [] #Lista com todas as palavras dos tweets relevantes

for i in lista_tweets_rel_completos:
    for w in i:
        palavras_tweets_rel.append(w)

serie_palavras_tweets_rel = pd.Series(palavras_tweets_rel) #Criando série para as palavras dos tweets relevantes

tabela_frequencias_tweets_rel = serie_palavras_tweets_rel.value_counts() #Tabela de frequencias absolutas das palavras dos tweets relevantes

#### Tweets irrelevantes

In [21]:
lista_tweets_irel_completos = [] #Lista com cada tweet relevante dentro de uma lista

for tweet in tweets_irel_limpo:
    lista_tweets_irel_completos.append(tweet.split())

palavras_tweets_irel = [] #Lista com todas as palavras dos tweets irrelevantes

for i in lista_tweets_irel_completos:
    for w in i:
        palavras_tweets_irel.append(w)

serie_palavras_tweets_irel = pd.Series(palavras_tweets_irel) #Criando série para as palavras dos tweets irrelevantes

tabela_frequencias_tweets_irel = serie_palavras_tweets_irel.value_counts() #Tabela de frequencias absolutas das palavras dos tweets irrelevantes

#### Variáveis para o Smoothing

In [22]:
alpha = 0.75
v = 400000

total_rel = len(tabela_frequencias_tweets_rel)+ alpha*v  #Total de palavras nos tweets relevantes(com smoothing)

total_irel = len(tabela_frequencias_tweets_irel)+ alpha*v  #Total de palavras nos tweets irrelevantes(com smoothing)

total_palavras = sum(tabela_frequencias_tweets_irel)+sum(tabela_frequencias_tweets_rel)

#### Calculando Priors

In [23]:
prob_rel = tabela_frequencias_tweets_rel.sum()/(total_palavras) #Probabilidade relevante
prob_irel = tabela_frequencias_tweets_irel.sum()/(total_palavras) #Probabilidade irrelevante

#### Função do Classificador Naive Bayes 


In [24]:
def Classificador(tweets, df): #Função do classificador
    
    prob_Tweet_dado_rel = 1.0 
    prob_Tweet_dado_irel = 1.0 
    cont = 0 
   
    for tweet in tweets: 
        
        for palavra in tweet: 
            
            if palavra in tabela_frequencias_tweets_rel: 
                prob_palavra_dado_rel = (tabela_frequencias_tweets_rel[palavra] + alpha) / total_rel
                prob_Tweet_dado_rel += log10(prob_palavra_dado_rel)
            else: 
                prob_palavra_dado_rel = (alpha) / total_rel
                prob_Tweet_dado_rel += log10(prob_palavra_dado_rel)
            
            if palavra in tabela_frequencias_tweets_irel: 
                prob_palavra_dado_irel = (tabela_frequencias_tweets_irel[palavra] + alpha) / total_irel
                prob_Tweet_dado_irel += log10(prob_palavra_dado_irel)
            else: 
                prob_palavra_dado_irel = (alpha) / total_irel
                prob_Tweet_dado_irel += log10(prob_palavra_dado_irel)  


        
        prob_rel_dado_tweet = prob_Tweet_dado_rel+log10(prob_rel)
        prob_irel_dado_tweet = prob_Tweet_dado_irel+log10(prob_irel) 
        
        if prob_rel_dado_tweet > prob_irel_dado_tweet:
            df.iloc[cont,2] = 1
        elif prob_rel_dado_tweet < prob_irel_dado_tweet:
            df.iloc[cont,2] = 0
        
        cont +=1
        prob_Tweet_dado_rel = 1.0
        prob_Tweet_dado_irel = 1.0 

#### Rodando o classificador na base de treinamento

In [25]:
tweets_treinamento = [] #Lista com todos os tweets de treinamento

treino_limpo = train['Treinamento'].apply(limpa)

for tweet in (treino_limpo):
    tweets_treinamento.append(tweet.split())
    
train['Classificador'] = 'NaN' #Nova coluna no dataframe

Classificador(tweets_treinamento, train) #Rodando classificador no treinamento

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

#### Preparando a base de Testes e rodando o Classificador

In [26]:
tweets_test_limpo = test['Teste'].apply(limpa) #Limpando tweets do teste 

tweets_teste = [] #Lista com os tweets de teste
for tweet in tweets_test_limpo:
    tweets_teste.append(tweet.split())


test['Classificador'] = 'NaN' #Nova coluna no dataframe

Classificador(tweets_teste, test) #Rodando classificador nos testes

#### Facilitando a visualização no dataFrame

In [27]:
train['Classificação'] = train['Classificação'].astype('category') 
train['Classificador'] = train['Classificador'].astype('category') 

test['Classificação'] = test['Classificação'].astype('category') 
test['Classificador'] = test['Classificador'].astype('category') 


train['Classificação'].cat.categories = ['Irrelevante', 'Relevante']
train['Classificador'].cat.categories = ['Irrelevante', 'Relevante'] 

test['Classificação'].cat.categories = ['Irrelevante', 'Relevante'] 
test['Classificador'].cat.categories = ['Irrelevante', 'Relevante']

#### Verificando a performace

In [28]:
treinamento = 0 #Variável que define quantos tweets foram classificados pelo classificador corretamente
for i in range(len(train)):
    classificacao = str(train.iloc[i, 1])
    classificador = str(train.iloc[i, 2])
    if classificador == classificacao:
        treinamento +=1       
    else: 
         pass

teste = 0 #Variável que define quantos tweets foram classificados pelo classificador corretamente
for i in range(len(test)):
    classificacao = str(test.iloc[i, 1])
    classificador = str(test.iloc[i, 2])
    if classificador == classificacao:
        teste +=1       
    else: 
         pass

print('Taxa de acerto da base de Treinamento: {:.2f}%'.format((treinamento/len(train['Classificação']))*100))
print('Taxa de acerto da base de Testes: {:.2f}%'.format((teste/len(test['Classificação']))*100))

Taxa de acerto da base de Treinamento: 95.13%
Taxa de acerto da base de Testes: 71.76%


#### Tabela para verificar performace

In [29]:
pd.crosstab(test['Classificação'], test['Classificador'], normalize = 'all')

Classificador,Irrelevante,Relevante
Classificação,,
Irrelevante,0.384259,0.092593
Relevante,0.189815,0.333333


___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**